In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
filename = '../input/credit-card-customers/BankChurners.csv'
data = pd.read_csv(filename)
data.head()

In [ ]:
data.shape

In [ ]:
#check the column-wise distribution of null values
data.isnull().values.sum()

In [ ]:
#Drop Unneccessary Columns( LAST 2 COLUMNS ) + ClientNum 

data = data.iloc[:,1:-2]
data.head()

In [ ]:
# Check the dtype 
categories = dict()
for col in data.columns:
    if data[col].dtype == 'object':
        categories[col] = data[col].unique()

In [ ]:
categories.keys()

In [ ]:
for col in categories.keys():
    print(col)
    print(data[col].value_counts(),"\n")

In [ ]:
# Representing the complete data Recod on Customer_id 1
# For understand the values for specific columns and generalize important keys on real-world understanding
data.loc[0]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
sns.pairplot(data,hue='Attrition_Flag')

In [ ]:
n = round(len(categories.keys())/2)
i=1
for col in categories.keys():    
    plt.subplot(n, 2, i)
    plt.tight_layout()
    plt.title(col)
    data[col].value_counts().plot(kind='bar',figsize=(15,10))
    i+=1    

In [ ]:
n = round(len(categories.keys())/2)
i=1

plt.figure(figsize=(16, 10))
for col in categories.keys():    
    if col != 'Attrition_Flag':
        plt.subplot(n, 2, i)
        plt.tight_layout()
        plt.title(col)
        sns.countplot(data[col],hue=data['Attrition_Flag'])
        #data[col].value_counts().plot(kind='bar'column,figsize=(15,10))
        i+=1   
plt.subplot(n, 2, i)
data['Attrition_Flag'].value_counts().plot(kind="pie",legend=True)

In [ ]:
#handling Categorical Data
df_cat = data[categories.keys()]
df_cat.head()

In [ ]:
df_cat = data[categories.keys()]
# Chage the Attrition_Flag entries 0 : Existing_Customer  1 : Attrited Customer
df_cat['Attrition_Flag'] =[0 if x =='Existing Customer' else 1 for x in data['Attrition_Flag']] #Also df_cat['Attrition_Flag'].replace({'Existing_CustomerF': 0, 'Attrited Customer':1})

# Chage the Gender entries-> 0: Male 1: Female

df_cat['Gender'] = [ 0 if x =='M' else 1 for x in data['Gender']] #Also df_cat['Gender'].replace({'F':1, 'M':0})

df_cat['Education_Level'] = df_cat['Education_Level'].replace({'Unknown' : 0, 
                                    'Uneducated': 1, 
                                    'High School' : 2, 
                                    'College' : 3, 
                                    'Graduate' : 4, 
                                    'Post-Graduate': 5, 
                                    'Doctorate' : 6})

df_cat['Income_Category'] = df_cat['Income_Category'].replace({'Unknown': 0,
                                                                 'Less than $40K' : 1,
                                                                 '$40K - $60K'    : 2,
                                                                 '$80K - $120K':  3,
                                                                 '$60K - $80K': 4,
                                                                 '$120K +': 5})

df_cat = pd.concat([df_cat,pd.get_dummies(df_cat['Card_Category'])],axis=1).drop(columns=['Card_Category','Blue'])
df_cat = pd.concat([df_cat,pd.get_dummies(df_cat['Marital_Status'])],axis=1).drop(columns=['Marital_Status','Unknown'])

In [ ]:
columns = ['Attrition_Flag', 'Gender', 'Education_Level', 'Marital_Status', 'Income_Category', 'Card_Category']
data = data.drop(columns,axis=1)
final_data = pd.concat([data,df_cat],axis=1)
final_data.head()

In [ ]:

plt.figure(figsize=(20,10))
sns.heatmap(final_data.corr(),annot=True,fmt='.1g')

## Applying & Prepare train,test data

In [ ]:
X = final_data.drop(columns=['Attrition_Flag'],axis=1)
y = final_data['Attrition_Flag']

from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [ ]:
print("Training Dataset",x_train.shape)
print("Testing Dataset",x_test.shape)

# Simply Model Training

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,roc_auc_score, plot_confusion_matrix #,confusion_matrix

def train_model(x_train,y_train,x_test,y_test):
    
    model = RandomForestClassifier(n_estimators=10)
    model.fit(x_train,y_train)
    
    predict = model.predict(x_test)
    
    print(classification_report(y_test,predict))
    #table = classification_report(y_test,predict,output_dict=True)
    #print(pd.DataFrame.from_dict(table))

    print("\nAUC Score",roc_auc_score(y_test,predict))

    #print(confusion_matrix(y_test,predict))
    plot_confusion_matrix(model,x_test,y_test)


In [ ]:
train_model(x_train,y_train,x_test,y_test)

### Accuracy of Simple Model Training:  0.94  

## Trying Features Selection method

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier()
model.fit(X,y)

#feat_importance = pd.Series(model.feature_importances_,index=X.columns)
#feat_importance.nlargest(5).plot(kind="barh")


In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif

selector = SelectKBest(f_classif, k=5)
X_selected = selector.fit_transform(x_train, y_train)

selected_features = pd.DataFrame(selector.inverse_transform(X_selected),columns=final_data.columns.drop('Attrition_Flag'))
selected_columns = selected_features.columns[selected_features.var() != 0]
# Get the valid dataset with the selected features.
final_data[selected_columns].head()

In [ ]:
# Apply the top 5 feature_columns to trained the model
train_model(x_train[selected_columns],y_train,x_test[selected_columns],y_test)

### Accuracy after Feature Selection Method: 0.89 {not good}

## Use Classifcation Model: RandomForestClassifier()

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(RandomForestClassifier(), x_train, y_train, cv=5, scoring='accuracy')
scores.mean()

### Parameter Tuning:RandomForestClassifier()

In [ ]:
model_rf = RandomForestClassifier()
n_estimators = [int(x) for x in np.linspace(100,1000,10)]
max_features = ['auto','sqrt']
max_depth = [int(x) for x in np.linspace(5, 30, 6)]
min_samples_split = [2, 5, 10, 15, 100]
min_samples_leaf = [1, 2, 5, 10]

params ={
            'n_estimators': n_estimators,
            'max_features': max_features,
            'max_depth': max_depth,
            'min_samples_split': min_samples_split,
            'min_samples_leaf': min_samples_leaf,
            'criterion' : ['gini','entropy']
        }

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

model_rscv = RandomizedSearchCV(estimator = model_rf, param_distributions = params,scoring='f1',random_state=42, n_jobs = 1)
model_rscv.fit(x_train,y_train)

print("Best Param: ", model_rscv.best_params_)
print("Best Score: ",model_rscv.best_score_)

### trained model with rfc_model.best_params_

In [ ]:
rfc_model = RandomForestClassifier(n_estimators= 700, min_samples_split=10, min_samples_leaf= 1, max_features='auto', max_depth=30, criterion='entropy').fit(x_train,y_train)

rfc_predict = rfc_model.predict(x_test)

print("Classification Report: \n",classification_report(y_test,rfc_predict))
print("AUC Score: ",roc_auc_score(y_test,rfc_predict))
plot_confusion_matrix(rfc_model,x_test,y_test)

## Use Classification Model: XGBClassifier()

In [ ]:
from xgboost import XGBClassifier

scores = cross_val_score(XGBClassifier(), x_train, y_train, cv=5, scoring='accuracy')
scores.mean()

### Parameter Tuning: XGBClassifier()

In [ ]:
param_grid = [
  {'colsample_bytree': [0.7, 0.8, 0.9, 1],
   'learning_rate': [0.3, 0.4, 0.5, 0.6, 0.7, 0.8],
   'max_depth': [4, 5, 6, 7, 8],
   'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100],
   'reg_lambda': [0.1, 1.0, 5.0, 10.0, 50.0, 100.0]
  }
]

In [ ]:
model_xgb = RandomizedSearchCV(estimator =XGBClassifier(), param_distributions = param_grid,scoring='f1',random_state=42, n_jobs = 1)
model_xgb.fit(x_train,y_train)

print("Best Param: ",model_xgb.best_params_)
print("Best Score: ",model_xgb.best_score_)

#predict_xgb = model_xgb.predict(x_test)
#print(classification_report(y_test,predict_rscv))
#print("AUC Score: ",roc_auc_score(y_test,predict_rscv))

### trained model with xgb_model.best_params_

In [ ]:
xgb_model = XGBClassifier(reg_lambda=50.0, reg_alpha= 1e-05, max_depth= 7, learning_rate=0.5,colsample_bytree=0.8).fit(x_train, y_train)

#scores = cross_val_score(xgb_model, x_train, y_train, cv=5, scoring='accuracy')
#scores.mean()

xgb_predict = xgb_model.predict(x_test)

print("Classification Report: \n",classification_report(y_test,xgb_predict))
print("AUC Score: ",roc_auc_score(y_test,xgb_predict))
plot_confusion_matrix(xgb_model,x_test,y_test)


# Accuracy: 0.97 using XGBClassifier()